# Generic

Helpers. All copied from previous notebooks

In [2]:
import math
import base64
import binascii
import numpy as np
import matplotlib.pyplot as plt

def to_bytes(d, format):
    if isinstance(d, (bytes, bytearray)):
        return d
    elif format == 'hex':
        return bytes(bytearray.fromhex(d))
    elif format == 'base64':
        return base64.b64decode(d)
    elif format == 'str' or format == 'bytes':
        return d.encode()
    elif format == 'int':
        return to_bytes(hex(d)[2:], 'hex')

def bytes_to(b, format):
    if not isinstance(b, (bytes, bytearray)):
        return b
    elif format == 'hex':
        return binascii.hexlify(b).decode()
    elif format == 'base64':
        return base64.b64encode(b).decode()
    elif format == 'str':
        return b.decode()
    elif format == 'bytes':
        return b
    elif format == 'int':
        return int(bytes_to(b, 'hex')[2:], 16)
    
def draw_bytestring(bs):
    """Represent a byte string as a matrix of color
    """
    tmp = bs + bytes(math.ceil(math.sqrt(len(bs)))**2 - len(bs))
    tmp = [int(b) for b in tmp]
    m = [[tmp[i:i+int(math.sqrt(len(tmp)))] for i in range(0, len(tmp), int(math.sqrt(len(tmp))))]]
    matrix = np.matrix(m[0])
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_aspect('equal')
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.binary)
    plt.colorbar()
    plt.show()

# Crypto

Simple crypto functions that are only accessory. Copied from previous notebooks.

In [3]:
import itertools
from Crypto.Cipher import AES

def pad(text, blocksize=16):
    padsize = blocksize - (len(text) % blocksize)
    if padsize == 0:
        padsize = blocksize
    return text + bytes([padsize]*padsize)
assert pad(b"YELLOW SUBMARINE", blocksize=20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'
def unpad(text):
    """Validate padding and return unpadded value"""
    assert text[-text[-1]:] == bytes([text[-1]]*text[-1])
    return text[:-text[-1]]
assert unpad(pad(b"this is random text", blocksize=7)) == b"this is random text"

def xor(d1, d2, format):
    b1,b2 = to_bytes(d1, format), to_bytes(d2, format)
    return bytes_to(bytes(char1 ^ char2 for char1,char2 in zip(b1,b2)), format)

def decrypt_aes_ecb(cipher, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        return unpad(aes.decrypt(cipher))
    else:
        return aes.decrypt(cipher)
def encrypt_aes_ecb(clear, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        clear = pad(clear)
    return aes.encrypt(clear)
assert decrypt_aes_ecb(encrypt_aes_ecb(b'test', 'YELLOW SUBMARINE'), 'YELLOW SUBMARINE') == b'test'


def encrypt_aes_cbc(clear, key, IV=None, debug=False):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = pad(clear, blocksize=16)
    prev_block = IV
    cipher = b''
    for blockstart in range(0, len(clear), 16):
        block = clear[blockstart:blockstart+16]
        if debug:
            print("encrypting...", block)
        cipher_block = encrypt_aes_ecb(xor(prev_block, block, 'bytes'), key, use_padding=False)
        prev_block = cipher_block
        cipher += cipher_block
    if debug:
        print("resulting cipher :", cipher)
    return cipher
def decrypt_aes_cbc(cipher, key, IV=None, debug=False):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = b''
    prev_block = IV
    for blockstart in range(0, len(cipher), 16):
        block = cipher[blockstart:blockstart+16]
        if debug:
            print("decrypting...", block)
        tmp = decrypt_aes_ecb(block, key, use_padding=False)
        clear_block = xor(prev_block, tmp, 'bytes')
        prev_block = cipher[blockstart:blockstart+16]
        clear += clear_block
    if debug:
        print("resulting clear :", clear)
    return unpad(clear)

test_iv = os.urandom(16)
assert decrypt_aes_cbc(encrypt_aes_cbc(b"cookingMCslikeapoundofbacon", b'YELLOW SUBMARINE', IV=test_iv), b'YELLOW SUBMARINE', IV=test_iv) == b"cookingMCslikeapoundofbacon"

    

In [4]:
pad(b"cookingMCslikeapoundofbacon", blocksize=16)

b'cookingMCslikeapoundofbacon\x05\x05\x05\x05\x05'

# Cryptopals Notebook

## Set 5

### Exercice 33

Diffie-Hellman.

In [5]:
import random
import hashlib

class DH:
    
    def __init__(self, p=37, g=5, a=None):
        self._p, self._g = p, g
        self._A, self._B, self._s = None, None, None
        self.generate_private_key(a)
        
    def generate_private_key(self, a=None):
        self._a = random.randint(0, self._p-1)
        if a is not None:
            self._a = a
        # goooo, python! power-mod is included
        # otherwise - not that hard to implement :
        # use a classic power algorithm but mod
        # the result everytime you multiply by the base
        self._A = pow(self._g, self._a, self._p)
        
    @property
    def public_key(self):
        if self._A is None:
            self.generate_private_key()
        return self._A

    def generate_session_key(self, B):
        # Generate the session key from the distant public key
        # and the local private key
        self._B = B
        self._s = pow(B, self._a, self._p)
        return self._s
        
    @property
    def session_key(self):
        return self._s
    
    @property
    def key(self):
        sha1 = hashlib.sha1()
        sha1.update(hex(self._s)[2:].encode())
        return sha1.digest()

In [6]:
alice = DH()
bob = DH()
alice.generate_session_key(bob.public_key)
bob.generate_session_key(alice.public_key)
print('Alice session key :', alice.session_key)
print('Bob session key :', bob.session_key)
assert alice.session_key == bob.session_key

Alice session key : 1
Bob session key : 1


In [7]:
class NISTDH(DH):
    def __init__(self, a=None):
        self._p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
        self._g = 2
        super().__init__(self._p, self._g, a)

In [8]:
alice = NISTDH()
bob = NISTDH()
alice.generate_session_key(bob.public_key)
bob.generate_session_key(alice.public_key)
print('Alice session key :', alice.key)
print('Bob session key :', bob.key)
assert alice.session_key == bob.session_key

Alice session key : b'B\x9f\x81\x9b\xe2\x05\xb4$\xf2\xdd\x05\xe5\x8bL\xb7s]Qm\x1c'
Bob session key : b'B\x9f\x81\x9b\xe2\x05\xb4$\xf2\xdd\x05\xe5\x8bL\xb7s]Qm\x1c'


### Exercice 34

MITM on DH. Let the fun begin ! We will emulate network actors with a simple message passing class. Back and forth is handled using callbacks and predefined functions. Messages are arbitrary python objects/

In [9]:
class NetworkActor:
    
    def __init__(self, name, actions):
        self._log = []
        self._name = name
        self._actions = actions
        self._dh = NISTDH()
        self._action_idx = 0
        
    @property
    def name(self):
        return self._name
    
    def initiate(self, receiver):
        # Trigger first action
        action = self._actions[self._action_idx]
        self._action_idx = (self._action_idx + 1) % len(self._actions)
        action(self, receiver)
    
    def send(self, receiver, msg):
        receiver.receive(self, msg)
        
    def receive(self, sender, msg):
        print('[.]', sender.name, '->', self.name, '\n', msg, '\n\n')
        if self._actions:
            action = self._actions[self._action_idx]
            self._action_idx = (self._action_idx + 1) % len(self._actions)
            action(self, sender, msg)

In [10]:
# First, the sequence without the MITM actor
import os

def ex34_initiate(alice, bob):
    # First, Alice sends its parameters to Bob
    alice._dh = NISTDH()
    bob.receive(alice, {"p": alice._dh._p, "g": alice._dh._g, "A": alice._dh.public_key})

def ex34_m1(bob, alice, msg):
    # Then bob replies with its public key
    p, g, A = msg["p"], msg["g"], msg["A"]
    bob._dh = DH(p, g)
    bob._dh.generate_session_key(A)
    bob.send(alice, {"B": bob._dh.public_key})

def ex34_m2(alice, bob, msg):
    # Alice replies with encrypted random message
    B = msg["B"]
    alice._dh.generate_session_key(B)
    payload = b"cookingMCslikeapoundofbacon"
    iv = os.urandom(16)
    key = alice._dh.key[:16]
    
    # Alice expects the same message in reply, but encrypted with
    # a different IV
    alice.expected_reply = payload
    alice.iv_to_avoid = iv
    
    encrypted_msg = encrypt_aes_cbc(payload, key, IV=iv)
    alice.send(bob, encrypted_msg + iv)
    
def ex34_m3(bob, alice, msg):
    # Bob decrypts Alice's message and re-encrypts it with another IV
    payload = msg[:-16]
    iv = msg[-16:]
    key = bob._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    new_iv = os.urandom(16) # don't check if it's the same. YOLO !
    encrypted_msg = encrypt_aes_cbc(cleartext, key, IV=new_iv)
    bob.send(alice, encrypted_msg + new_iv)
    
def ex34_final(alice, _, msg):
    # Alice decrypts incoming message and checks that it is the same
    payload = msg[:-16]
    iv = msg[-16:]
    key = alice._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    assert iv != alice.iv_to_avoid
    assert cleartext == alice.expected_reply
    

Alice = NetworkActor("Alice", actions=[ex34_initiate, ex34_m2, ex34_final])
Bob = NetworkActor("Bob", actions=[ex34_m1, ex34_m3])

Alice.initiate(Bob)

[.] Alice -> Bob 
 {'A': 345345052776855862231672304147076359450287635878727296474557362617007588025852310012119428486330759163979380674655784337964244549497524125571143166843597889199559810671707394269832127434194217847288364307031947228566913688948804490779208692286657831432520535886690631917267529388084708744374813461057750727338680756643695418664934098464635862758084562766575300005540585284248869065885874892345276380801368843961380781297596420260057377534359147760351877497610244, 'g': 2, 'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919} 


[.] Bob -> Alice 
 {'B': 14662

$$For\ B:\\\begin{eqnarray}
s &=& (A ^ b)\mod p\\
&=& (p ^ b)\mod p\\
&=& 0\\\end{eqnarray}$$

$$For \ A:\\\begin{eqnarray}
s &=& (B ^ a)\mod p\\
&=& (p ^ a)\mod p\\
&=& 0\end{eqnarray}$$

In [11]:
# Now, the sequence with the attacker (Eve)

class MITM(NetworkActor):
    def __init__(self, alice, bob, name, actions):
        self.alice = alice
        self.bob = bob
        super().__init__(name, actions)

def ex34_initiate_i(eve, _, msg):
    # Eve alters the message !
    # Bob still should think the message came from Alice in a real world attack
    eve.intercepted_p, eve.intercepted_g, eve.intercepted_A = msg["p"], msg["g"], msg["A"]
    eve.bob.receive(eve, {"p": msg["p"], "g": msg["g"], "A": msg["p"]})
    
    
def ex34_m1_i(eve, _, msg):
    # Eve also modifies this one
    eve.intercepted_B = msg["B"]
    eve.alice.receive(eve, {"B": eve.intercepted_p})
    
def ex34_m2_i(eve, _, msg):
    # Eve intercepts the message and decrypts it easily
    sha1 = hashlib.sha1()
    sha1.update(hex(0)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_1 = msg
    eve.cleartext_1 = cleartext
    eve.bob.receive(eve, msg)
    
def ex34_m3_i(eve, _, msg):
    # confirm with the return message
    sha1 = hashlib.sha1()
    sha1.update(hex(0)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_2 = msg
    eve.cleartext_2 = cleartext
    assert eve.cleartext_1 == eve.cleartext_2
    print('Intercepted message', cleartext, "\n\n")
    
    eve.alice.receive(eve, msg)
    

Eve = MITM(Alice, Bob, "Eve", actions=[ex34_initiate_i, ex34_m1_i, ex34_m2_i, ex34_m3_i])

Alice.initiate(Eve)

[.] Alice -> Eve 
 {'A': 443049588704565391313195587147096184433398399479170434082528432503023842926166435478670653739612126534230211246866798329998219529711126519340623783011461785078592572808449228619274481038712441765920703415103887004410893644335378181920012868614185903571651881397694149052092223845158829095894831438899022063973117753118316927551138984867590106982993306577427475937358989731003855309595913422040193985832677183333735758661693503371578160452717301272404293175359159, 'g': 2, 'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919} 


[.] Eve -> Bob 
 {'A': 2410312

### Exercice 35

Same idea but with manipulating g instead of the public keys.

In [12]:
# The base-case without the MITM

def ex35_initiate(alice, bob):
    # All messages now go through Eve
    alice._dh = NISTDH()
    alice._sent_p, alice._sent_g = alice._dh._p, alice._dh._g
    bob.receive(alice, {"p": alice._dh._p, "g": alice._dh._g})
    
def ex35_m1(bob, alice, msg):
    # Respond with ACK
    p, g = msg["p"], msg["g"]
    bob._dh = DH(p, g)
    alice.receive(bob, {"g": g, "ACK": "ACK"})
    
def ex35_m2(alice, bob, msg):
    assert msg["ACK"] == "ACK"
    
    # The exercice is worded pretty loosely for this one.
    # We just assume that we can change the g value of both parties
    # else there is a key mismatch in the end. If we don't assume
    # this, we could always renegotiate public keys with Alice for
    # the MITM attack, but that kinda defeats the purpose of giving
    # Bob a manipulated g value.
    alice._dh = DH(alice._sent_p, msg["g"])
    alice._dh.generate_private_key()
    bob.receive(alice, alice._dh.public_key)
    
def ex35_m3(bob, alice, msg):
    bob._dh.generate_session_key(msg)
    alice.receive(bob, bob._dh.public_key)

def ex35_m4(alice, bob, msg):
    # Alice replies with encrypted random message
    B = msg
    alice._dh.generate_session_key(B)
    payload = b"cookingMCslikeapoundofbacon"
    iv = os.urandom(16)
    key = alice._dh.key[:16]
    
    # Alice expects the same message in reply, but encrypted with
    # a different IV
    alice.expected_reply = payload
    alice.iv_to_avoid = iv
    
    encrypted_msg = encrypt_aes_cbc(payload, key, IV=iv)
    alice.send(bob, encrypted_msg + iv)
    
def ex35_m5(bob, alice, msg):
    # Bob decrypts Alice's message and re-encrypts it with another IV
    payload = msg[:-16]
    iv = msg[-16:]
    key = bob._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    new_iv = os.urandom(16) # don't check if it's the same. YOLO !
    encrypted_msg = encrypt_aes_cbc(cleartext, key, IV=new_iv)
    bob.send(alice, encrypted_msg + new_iv)
    
def ex35_final(alice, _, msg):
    # Alice decrypts incoming message and checks that it is the same
    payload = msg[:-16]
    iv = msg[-16:]
    key = alice._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    assert iv != alice.iv_to_avoid
    assert cleartext == alice.expected_reply

Alice = NetworkActor("Alice", actions=[ex35_initiate, ex35_m2, ex35_m4, ex35_final])
Bob = NetworkActor("Bob", actions=[ex35_m1, ex35_m3, ex35_m5])

Alice.initiate(Bob)

[.] Alice -> Bob 
 {'g': 2, 'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919} 


[.] Bob -> Alice 
 {'ACK': 'ACK', 'g': 2} 


[.] Alice -> Bob 
 193992475175359131955686746849896181453506938230334565570788859312231740382419978268078168545589387895629680515934773358007404602350584501941384544413364004300041448959753413306250454282465383269531387700335060704448090075860291186010365401101415158901273664525798162379759963300011548170645350166014770872253639557457912493416176471896790608176357557835956835977641441037961170716851285814498449630783524488170622306818663976279637349

All the different scenarios :

$$With\ g=1:\\\begin{eqnarray}
A &=& (1 ^ a)\mod p\\
&=& 1\\
\implies s &=& (1 ^ a)\mod p\\
&=& 1\\\end{eqnarray}$$

$$With\ g=p:\\\begin{eqnarray}
A &=& (p ^ a)\mod p\\
&=& 0\\
\implies s &=& (0 ^ a)\mod p\\
&=& 0\\\end{eqnarray}$$

$$With\ g=p-1:\\\begin{eqnarray}
A &=& ((p-1) ^ a)\mod p\\
&=& 1\\
\implies s &=& (1 ^ a)\mod p\\
&=& 1\\\end{eqnarray}$$

In [13]:
# Aaaand with the MITM
    
def ex35_initiate_i(eve, _, msg):
    # Eve intercepts this message and changes the g value
    eve.intercepted_p, eve.intercepted_g = msg["p"], msg["g"]
    fake_msg = {"p": msg["p"], "g": eve.fake_g}
    eve.bob.receive(eve, fake_msg)
    
def passthrough_bob_to_alice(eve, _, msg):
    # Generic passthrough method
    print("generic b>a")
    eve.alice.receive(eve, msg)
    
def passthrough_alice_to_bob(eve, _, msg):
    # Generic passthrough method
    print("generic a>b")
    eve.bob.receive(eve, msg)
    
def ex35_m4_i(eve, _, msg):
    # Eve gets the encrypted message, but already knows the key
    sha1 = hashlib.sha1()
    sha1.update(hex(eve.forced_key)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    print("key", key)
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_1 = msg
    eve.cleartext_1 = cleartext
    eve.bob.receive(eve, msg)
    
def ex35_m5_i(eve, _, msg):
    # confirm with the return message
    sha1 = hashlib.sha1()
    sha1.update(hex(eve.forced_key)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_2 = msg
    eve.cleartext_2 = cleartext
    assert eve.cleartext_1 == eve.cleartext_2
    print('Intercepted message', cleartext)
    
    eve.alice.receive(eve, msg)
    
Eve = MITM(Alice, Bob, "Eve", actions=[ex35_initiate_i, passthrough_bob_to_alice,
                                       passthrough_alice_to_bob,
                                       passthrough_bob_to_alice,
                                       ex35_m4_i, ex35_m5_i])


In [14]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff

# g = 1
Eve.fake_g, Eve.forced_key = 1, 1
Alice.initiate(Eve)

# g = p - 1
Eve.fake_g, Eve.forced_key = p-1, 1
Alice.initiate(Eve)

# g = p
Eve.fake_g, Eve.forced_key = p, 0
Alice.initiate(Eve)

[.] Alice -> Eve 
 {'g': 2, 'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919} 


[.] Eve -> Bob 
 {'g': 1, 'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919} 




### Exercice 36

Implement SRP

In [23]:
import os
import random

def sha256(msg):
    hasher = hashlib.sha256()
    hasher.update(msg.encode())
    return hasher.digest()

def sha256mac(key, message):
    return sha256(key + message) # Meh. Good enough :p.

class SRPServer:
    def __init__(self, email, password,
                 N=0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff,
                 g=2, k=3):
        # Init "hash" of password
        self._b = random.randint(0, N-1)
        self._k, self._N, self._g = k, N, g
        self._I = email
        self._salt = random.randint(0, 2**32)
        xH = sha256(hex(self._salt)[2:] + password)
        x = bytes_to(xH, 'int')
        self._v = pow(g, x, N)
        self._B = (k * self._v) % N + pow(g, self._b, N)
        
    def receive_msg1(self, msg):
        # Client "initiates" authentication flow
        self._A = msg['A']
    
    def send_msg2(self, rcvr):
        # Server transmits salt and his "public" key
        print('[Bob] > [Alice]:\n{}'.format({'B': self._B, 's': self._salt}))
        rcvr.receive_msg2({'B': self._B, 's': self._salt})
        
    def receive_msg3(self, msg):
        # First, calculate shared "hash"
        uH = sha256(hex(self._A)[2:] + hex(self._B)[2:])
        u = bytes_to(uH, 'int')
        b = self._b
        S = pow(self._A * pow(self._v, u, self._N), b, self._N)
        print('S Server', S)
        K = sha256(hex(S)[2:])
        
        # Now, compare HMAC
        client_hmac = msg['hmac']
        hmac = sha256mac(str(K), hex(self._salt)[2:])
        print(hmac)
        return hmac == client_hmac

class SRPClient:
    def __init__(self, email, password,
                 N=0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff,
                 g=2, k=3):
        self._a = random.randint(0, N-1)
        self._A = pow(g, self._a, N)
        self._N, self._g, self._k = N, g, k
        self._email, self._password = email, password # Client knows the secrets
        
    def send_msg1(self, rcvr):
        # Client "initiates" authentication flow
        print('[Alice] > [Bob]:\n{}'.format({'A': self._A, 'I': self._email}))
        rcvr.receive_msg1({'A': self._A, 'I': self._email})
        
    def receive_msg2(self, msg):
        # Server transmits salt and his "public" key
        self._salt = msg['s']
        self._B = msg['B']
        
        # Now, calculate shared "hash"
        uH = sha256(hex(self._A)[2:] + hex(self._B)[2:])
        u = bytes_to(uH, 'int')
        xH = sha256(hex(self._salt)[2:] + self._password)
        x = bytes_to(xH, 'int')
        
        # Here come the actual keys
        a = self._a
        S = pow(self._g, self._a 
        print('S Client', S)
        self._K = sha256(hex(S)[2:])
    
    def send_msg3(self, rcvr):
        # Final authentication with the server
        print('[Alice] > [Bob]\n{}'.format({'hmac': sha256mac(str(self._K), hex(self._salt)[2:])}))
        assert rcvr.receive_msg3({'hmac': sha256mac(str(self._K), hex(self._salt)[2:])})
        
        

email = 'email'
password = 'password'
c = SRPClient(email, password)
s = SRPServer(email, password)
c.send_msg1(s)
s.send_msg2(c)
c.send_msg3(s)
print('Authenticated \o/ !') # If the authentication fails, there is an assertion error

[Alice] > [Bob]:
{'I': 'email', 'A': 476679335518616146427294484213762276452260658691137415062912659810097395004675874839261506125895450367560829425943474845493962844874907175862598554018139623464680154816682920269959213234697680572974924160996949412073558735424243370567878147543516606116189506351698602409617671172046902699356627441918834027848361802416986048365604875874064188036367603737976699149461529178080449188510152042530017293014298685490978463415551253664582189110684432002313588211335478}
[Bob] > [Alice]:
{'s': 521142072, 'B': 3349014697826049098487156289449821496912503962656864413735098178300352194281973343176083206943900342040798361653937290500996311654148891021431693834259227897767909010038645810746951539017527208964177684982658479990409430988624829520673687480516597955692614374413072679084753233249846276188658706272947347993589770661385748818085577570221990802844418538430788088096092830249146823177803169316586710001869266003467308093793735904257725845941630374437679963675298

AssertionError: 

In [32]:
"test".encode()

b'test'